# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [27]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import math
import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error 
import matplotlib.pyplot as plt

import gensim
from nltk.tokenize import WordPunctTokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Embedding

from tensorflow.python.keras.models import Model, Layer, Sequential
from tensorflow.python.keras.layers import LSTM, Bidirectional, Subtract, Input, Dense, Flatten, Multiply, Concatenate, MaxPooling1D, MaxPooling2D, MaxPooling3D, Dot
from keras.layers.core import Layer
import keras
import keras.backend as K

import gzip
import shutil

In [3]:
%notebook inline

In [4]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

# Constant

In [5]:
ROOT_PATH = '/content/drive/My Drive/13516085 - Muh Habibi Haidir';

# Helper Function

In [20]:
def changes_data_to_x(tokenizer, data):
    sequences_student = t.texts_to_sequences(data['jawaban_pelajar'])
    sequences_reference = t.texts_to_sequences(data['jawaban_referensi'])

    X_student = pad_sequences(sequences_student, maxlen=MAX_SEQUENCE_LENGTH)
    X_reference = pad_sequences(sequences_reference, maxlen=MAX_SEQUENCE_LENGTH)
    
    return X_student, X_reference

# Main

## Prepare Data

In [7]:
student_saintek_answer_train = pd.read_csv('{}/Code/Data/Indonesia/score-saintek.csv'.format(ROOT_PATH))
total_problem_saintek_train = len(np.unique(student_saintek_answer_train['type-problem']))

student_soshum_answer_train = pd.read_csv('{}/Code/Data/Indonesia/score-soshum.csv'.format(ROOT_PATH))
student_soshum_answer_train['type-problem'] = student_soshum_answer_train['type-problem'] + total_problem_saintek_train

question_saintek_answer_train = pd.read_csv('{}/Code/Data/Indonesia/question-saintek.csv'.format(ROOT_PATH))
question_soshum_answer_train = pd.read_csv('{}/Code/Data/Indonesia/question-soshum.csv'.format(ROOT_PATH))
question_soshum_answer_train['type-problem'] = question_soshum_answer_train['type-problem'] + total_problem_saintek_train

student_saintek_answer_test = pd.read_csv('{}/Code/Data/Indonesia/score-saintek-test.csv'.format(ROOT_PATH))
total_problem_saintek_test = len(np.unique(student_saintek_answer_test['type-problem']))

student_soshum_answer_test = pd.read_csv('{}/Code/Data/Indonesia/score-soshum-test.csv'.format(ROOT_PATH))
student_soshum_answer_test['type-problem'] = student_soshum_answer_test['type-problem'] + total_problem_saintek_test

question_saintek_answer_test = pd.read_csv('{}/Code/Data/Indonesia/question-saintek-test.csv'.format(ROOT_PATH))
question_soshum_answer_test = pd.read_csv('{}/Code/Data/Indonesia/question-soshum-test.csv'.format(ROOT_PATH))
question_soshum_answer_test['type-problem'] = question_soshum_answer_test['type-problem'] + total_problem_saintek_test

In [8]:
student_answer_train = pd.concat([student_saintek_answer_train, student_soshum_answer_train])
question_answer_train = pd.concat([question_saintek_answer_train, question_soshum_answer_train])

student_answer_test = pd.concat([student_saintek_answer_test, student_soshum_answer_test])
question_answer_test = pd.concat([question_saintek_answer_test, question_soshum_answer_test])

print('Student Answer Train Shape: {}'.format(student_answer_train.shape))
display(student_answer_train.head())
print('Question Answer Train Shape: {}'.format(question_answer_train.shape))
display(question_answer_train.head())

print('Student Answer Test Shape: {}'.format(student_answer_test.shape))
print('Question Answer Test Shape: {}'.format(question_answer_test.shape))

Student Answer Train Shape: (7526, 3)


,type-problem,jawaban,score
0,1,Bakteri penyubur tanaman,1
1,1,"Entahlah, yang jelas bakteri berplasmid",1
2,1,Bakteri baik Bakteri hasil mutasi.,1
3,1,Bakteri,0
4,1,"Pseudomonas syringae, agrobacterium tumefacien...",4


Question Answer Train Shape: (30, 3)


,type-problem,pertanyaan,jawaban
0,1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
1,2,Jelaskan perbedaan antara meristem primer deng...,Meristem primer terlibat dalam pertumbuhan pri...
2,3,Apa yang dimaksud dengan masa dormansi biji? A...,Dormansi biji adalah keadaan ketika biji mence...
3,4,Apakah yang dimaksud perkembangan?,Perkembangan adalah transformasi suatu orgnai...
4,5,Di daerah diferensiasi terdapat tiga sistem ja...,Jaringan dermal adalah perlindungan mencegah k...


Student Answer Test Shape: (1560, 3)
Question Answer Test Shape: (6, 3)


In [9]:
idx_problem_train_saintek = sklearn.utils.shuffle(np.arange(1, 16), random_state=21)
idx_problem_train_soshum = sklearn.utils.shuffle(np.arange(16, 31), random_state=21)

idx_problem_train = np.concatenate((idx_problem_train_saintek[3:], idx_problem_train_soshum[3:]))
idx_problem_val = np.concatenate((idx_problem_train_saintek[:3], idx_problem_train_soshum[:3])) 

In [10]:
data_train = pd.merge(student_answer_train, question_answer_train, how='inner', on='type-problem', suffixes=(['_pelajar', '_referensi']))
data_train = data_train.fillna('')
data_test = pd.merge(student_answer_test, question_answer_test, how='inner', on='type-problem', suffixes=(['_pelajar', '_referensi']))
data_test = data_test.fillna('')

data_val = data_train[data_train['type-problem'].isin(idx_problem_val)]
data_train = data_train[data_train['type-problem'].isin(idx_problem_train)]

y_test = data_test['score']/5
y_train = data_train['score']/5
y_val = data_val['score']/5

print('Data Train Shape: {}'.format(data_train.shape))
display(data_train.head())
print('Data Val Shape: {}'.format(data_val.shape))
display(data_val.head())
print('Data Test Shape: {}'.format(data_test.shape))
display(data_test.head())

Data Train Shape: (6005, 5)


,type-problem,jawaban_pelajar,score,pertanyaan,jawaban_referensi
0,1,Bakteri penyubur tanaman,1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
1,1,"Entahlah, yang jelas bakteri berplasmid",1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
2,1,Bakteri baik Bakteri hasil mutasi.,1,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
3,1,Bakteri,0,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"
4,1,"Pseudomonas syringae, agrobacterium tumefacien...",4,Sebutkan bakteri apa saja yang terlibat dalam ...,"Bacillus thuringiensis, Agrobacterium tumefaciens"


Data Val Shape: (1521, 5)


,type-problem,jawaban_pelajar,score,pertanyaan,jawaban_referensi
1116,6,Sel spons,1,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1117,6,Lupa juga,0,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1118,6,-,0,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1119,6,Tidak tahu,0,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."
1120,6,"Dermatogen, periblem, plerom",4,Sebutkan lapisan sel yang dikemukakan oleh Han...,"Plerom, periblem, dan dermatogen."


Data Test Shape: (1560, 5)


,type-problem,jawaban_pelajar,score,pertanyaan,jawaban_referensi
0,1,Ekosistem adalah kumpulan materi abiotik dan b...,3,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
1,1,suatu sistem ekologi yang terbentuk oleh hubun...,3,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
2,1,ekosistem adalah komunitas organisme makhluk h...,5,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
3,1,Ekosistem adalah satu kesatuan makhluk hidup d...,2,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...
4,1,Hubungan timbal balik antara mahkluk hidup den...,5,Apakah yang dimaksud dengan ekosistem?,ekosistem adalah hubungan timbal balik antara ...


## Model Baseline

### Constant

In [11]:
N_EPOCH_BASELINE = 15
N_BATCH_BASELINE = 10
N_HIDDEN_BASELINE = 50

EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 200

### Main

In [12]:
model_fasttext = gensim.models.fasttext.FastText.load_fasttext_format('{}/Code/Model/cc.id.300.bin'.format(ROOT_PATH))

In [16]:
# Get All Texts

texts_train = np.append(data_train['jawaban_pelajar'].values, data_train['jawaban_referensi'].values)
texts_val = np.append(data_val['jawaban_pelajar'].values, data_val['jawaban_referensi'].values)
texts_test = np.append(data_test['jawaban_pelajar'].values, data_test['jawaban_referensi'].values)
texts = np.append(texts_train, texts_val)
texts = np.append(texts, texts_test)

In [17]:
# Prepare tokenizer 

t = Tokenizer()
t.fit_on_texts(texts)

word_index = t.word_index
print('Found {} unique tokens'.format(len(word_index)))

Found 5446 unique tokens


In [18]:
# Prepare Embedding Layer

vocab_size = len(word_index)+1
print('Vocabulary Size: {}'.format(vocab_size))

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word_index.items():
  try:
    embedding_vector = model_fasttext[word]
    embedding_matrix[i] = embedding_vector
  except KeyError:
    embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
  
embedding_layer = Embedding(vocab_size, EMBEDDING_DIM, weights=[embedding_matrix])

Vocabulary Size: 5447


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [21]:
X_train_student, X_train_reference = changes_data_to_x(t, data_train)
X_val_student, X_val_reference = changes_data_to_x(t, data_val)
X_test_student, X_test_reference = changes_data_to_x(t, data_test)

print('Shape X_train_student: {}, X_train_reference: {}'.format(X_train_student.shape, X_train_reference.shape))
print('Shape X_val_student: {}, X_val_reference: {}'.format(X_val_student.shape, X_val_reference.shape))
print('Shape X_test_student: {}, X_test_reference: {}'.format(X_test_student.shape, X_test_reference.shape))

Shape X_train_student: (6005, 200), X_train_reference: (6005, 200)
Shape X_val_student: (1521, 200), X_val_reference: (1521, 200)
Shape X_test_student: (1560, 200), X_test_reference: (1560, 200)


### Linear Regression

#### Helper Function

In [44]:
def create_model_baseline(n_hidden=N_HIDDEN_BASELINE, scoring_type='multiply'):
    # Prepare Each Answer Model
    input_student =  Input(shape=(200,))
    input_reference = Input(shape=(200,))

    embedding_student = embedding_layer(input_student)
    bilstm_student = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='ave')(embedding_student)

    embedding_reference = embedding_layer(input_reference)
    bilstm_reference = Bidirectional(LSTM(n_hidden, return_sequences=True), merge_mode='ave')(embedding_reference)

    subtracted = Subtract()([bilstm_student, bilstm_reference])
    multiplied =  Multiply()([bilstm_student, bilstm_reference])
    cosine_similarity = Dot(axes=1, normalize=True)([bilstm_student, bilstm_reference])
    
    if scoring_type == 'nothing':
      concatenated = Concatenate()([bilstm_student, bilstm_reference])
    elif scoring_type == 'sub':
      concatenated = Concatenate()([bilstm_student, bilstm_reference, subtracted])
    elif scoring_type == 'multiply':
      concatenated = Concatenate()([bilstm_student, bilstm_reference, subtracted, multiplied])
    # elif scoring_type == 'cosine':
    #   concatenated = Concatenate()([tf.keras.backend.ones((50, 50)), cosine_similarity])
    flatted = Flatten()(concatenated)
    out = Dense(1, activation='sigmoid')(flatted)

    model_baseline = Model(inputs=[input_student, input_reference], outputs=out)
    model_baseline.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    
    return model_baseline

#### Main

In [45]:
scoring_types = ['nothing', 'sub', 'multiply']

In [ ]:
for scoring_type in scoring_types:
  model_baseline = create_model_baseline(50, scoring_type='sub')
  
  print(model_baseline.summary())

  # checkpoint_path = '{}/Code/Model/model-indonesia/word-embedding/indonesia.ckpt'.format(ROOT_PATH)

  # Create a callback that saves the model's weights
  # cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
  #                                                 save_weights_only=True,
  #                                                 verbose=1)
  
  model_baseline.fit([X_train_student, X_train_reference], y_train, validation_data=([X_test_student, X_test_reference], y_test), epochs=N_EPOCH_BASELINE, batch_size=N_BATCH_BASELINE)
  
  predicts_test_baseline = model_baseline.predict([X_test_student, X_test_reference])
  predicts_train_baseline = model_baseline.predict([X_train_student, X_train_reference])

  mae_train_baseline = mean_absolute_error(y_train, predicts_train_baseline)
  mse_train_baseline = mean_squared_error(y_train, predicts_train_baseline)
  mae_baseline = mean_absolute_error(y_test, predicts_test_baseline)
  mse_baseline = mean_squared_error(y_test, predicts_test_baseline)

  print('Scoring Type: {}'.format(scoring_type))

  print('Mean Absolute Error Train: {}'.format(mae_train_baseline))
  print('Mean Squared Error Train: {}'.format(mse_train_baseline))
  print('Mean Absolute Error: {}'.format(mae_baseline))
  print('Mean Squared Error: {}'.format(mse_baseline))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 300)     1634100     input_19[0][0]                   
                                                                 input_20[0][0]                   
__________________________________________________________________________________________________
bidirectional_18 (Bidirectional (None, 200, 50)      140400      embedding[18][0]           